This file is produced by Hengyi Li and Jingbo Wang

2023.3.10

# Fundamentals of machine learning

## Generalization: The goal of machine learning

We can setup our envrinment by importing those libraries 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import imdb

Before start, we can check our running environment by running those code

In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Improving generalization

To improve the generalization, we have several ways to do it. 

* Dataset curation

* Feature engineering

* Using early stopping

* Reducing the network's size

  * Reducing the network's size
  * Adding L2 weight regularization to the model
  * Adding dropout to the IMDB model

### Dataset curation

* Make sure you have enough data. Remember that you need a _**dense sampling**_ of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Make sure you have enough data. Remember that you need a dense sampling of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Clean your data and deal with missing values.
* If you have many features and you aren’t sure which ones are actually useful, do feature selection.


So the num_words restricted the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. And we can do better than that by increasing the num_words to 25000 

**Original Model**

In [3]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 5s 35ms/step - loss: 0.5351 - accuracy: 0.7859 - val_loss: 0.4194 - val_accuracy: 0.8540
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3299 - accuracy: 0.8967 - val_loss: 0.3341 - val_accuracy: 0.8718
Epoch 3/20
30/30 [==============================] - 1s 26ms/step - loss: 0.2405 - accuracy: 0.9228 - val_loss: 0.2904 - val_accuracy: 0.8851
Epoch 4/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1905 - accuracy: 0.9375 - val_loss: 0.2712 - val_accuracy: 0.8938
Epoch 5/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1528 - accuracy: 0.9512 - val_loss: 0.2996 - val_accuracy: 0.8791
Epoch 6/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1289 - accuracy: 0.9589 - val_loss: 0.2865 - val_accuracy: 0.8877
Epoch 7/20
30/30 [==============================] - 1s 25ms/step - loss: 0.1058 - accuracy: 0.9677 - val_loss: 0.3079 - val_accuracy: 0.8827
Epoch 8/20
30

**Practice Model**

Here, we change the num_words to 25000 

In [4]:
(train_data, train_labels), _ = imdb.load_data(num_words=25000)

def vectorize_sequences(sequences, dimension=25000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 74ms/step - loss: 0.5303 - accuracy: 0.7907 - val_loss: 0.3968 - val_accuracy: 0.8728
Epoch 2/20
30/30 [==============================] - 1s 40ms/step - loss: 0.2991 - accuracy: 0.9111 - val_loss: 0.3010 - val_accuracy: 0.8926
Epoch 3/20
30/30 [==============================] - 1s 46ms/step - loss: 0.2044 - accuracy: 0.9414 - val_loss: 0.2696 - val_accuracy: 0.8966
Epoch 4/20
30/30 [==============================] - 1s 35ms/step - loss: 0.1457 - accuracy: 0.9575 - val_loss: 0.2634 - val_accuracy: 0.8959
Epoch 5/20
30/30 [==============================] - 1s 43ms/step - loss: 0.1068 - accuracy: 0.9707 - val_loss: 0.2770 - val_accuracy: 0.8912
Epoch 6/20
30/30 [==============================] - 1s 35ms/step - loss: 0.0783 - accuracy: 0.9798 - val_loss: 0.2982 - val_accuracy: 0.8879
Epoch 7/20
30/30 [==============================] - 1s 46ms/step - loss: 0.0587 - accuracy: 0.9863 - val_loss: 0.3019 - val_accuracy: 0.8898
Epoch 8/20
30

As it shown above, after change the num_words and the batch size, make a bigger dataset and more training on each epoch, the accuracy gets visible improvement

### Feature engineering

**Original Model**

In [5]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 2s 35ms/step - loss: 0.5358 - accuracy: 0.7835 - val_loss: 0.4123 - val_accuracy: 0.8628
Epoch 2/50
30/30 [==============================] - 1s 20ms/step - loss: 0.3323 - accuracy: 0.8964 - val_loss: 0.3178 - val_accuracy: 0.8834
Epoch 3/50
30/30 [==============================] - 1s 20ms/step - loss: 0.2397 - accuracy: 0.9212 - val_loss: 0.2820 - val_accuracy: 0.8913
Epoch 4/50
30/30 [==============================] - 1s 23ms/step - loss: 0.1855 - accuracy: 0.9391 - val_loss: 0.2756 - val_accuracy: 0.8921
Epoch 5/50
30/30 [==============================] - 1s 24ms/step - loss: 0.1509 - accuracy: 0.9523 - val_loss: 0.2904 - val_accuracy: 0.8859
Epoch 6/50
30/30 [==============================] - 1s 23ms/step - loss: 0.1251 - accuracy: 0.9614 - val_loss: 0.2892 - val_accuracy: 0.8877
Epoch 7/50
30/30 [==============================] - 1s 24ms/step - loss: 0.1058 - accuracy: 0.9676 - val_loss: 0.3032 - val_accuracy: 0.8867
Epoch 8/50
30

**Practice Model**

We can enhance by deleting the most common words

In [6]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)
common_stop_words=["a","an","and","the","in","of","to","that","this","for"]


def delete_common_stop_words(train_data, common_stop_words):
    word_index = imdb.get_word_index()
    for i in range(len(common_stop_words)):
      if word_index[common_stop_words[i]]:
         common_stop_word_index = word_index[common_stop_words[i]]
         for i in range(len(train_data)):
           train_data[i] = list(
               filter((common_stop_word_index).__ne__, train_data[i]))
    return train_data

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = delete_common_stop_words(train_data, common_stop_words)
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
Practice_Run  = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 2s 34ms/step - loss: 0.5517 - accuracy: 0.7858 - val_loss: 0.4332 - val_accuracy: 0.8635
Epoch 2/50
30/30 [==============================] - 1s 19ms/step - loss: 0.3465 - accuracy: 0.8997 - val_loss: 0.3267 - val_accuracy: 0.8849
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 0.2513 - accuracy: 0.9208 - val_loss: 0.2872 - val_accuracy: 0.8890
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 0.1954 - accuracy: 0.9379 - val_loss: 0.2953 - val_accuracy: 0.8813
Epoch 5/50
30/30 [==============================] - 1s 24ms/step - loss: 0.1591 - accuracy: 0.9502 - val_loss: 0.2843 - val_accuracy: 0.8865
Epoch 6/50
30/30 [==============================] - 1s 23ms/step - loss: 0.1307 - accuracy: 0.9605 - val_loss: 0.2978 - val_accuracy: 0.8845
Epoch 7/50
30/30 [==============================] - 1s 23ms/step - loss: 0.1096 - accuracy: 0.9667 - val_loss: 0.3050 - val_accuracy: 0.8853
Epoch 8/50
30

### Using early stopping

We could using early stopping class in the Keras library to accomplish this step

**Original model**

In [7]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 2s 35ms/step - loss: 0.5408 - accuracy: 0.7583 - val_loss: 0.3947 - val_accuracy: 0.8623
Epoch 2/50
30/30 [==============================] - 1s 20ms/step - loss: 0.3139 - accuracy: 0.8989 - val_loss: 0.3027 - val_accuracy: 0.8884
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.2228 - accuracy: 0.9271 - val_loss: 0.3199 - val_accuracy: 0.8742
Epoch 4/50
30/30 [==============================] - 1s 22ms/step - loss: 0.1695 - accuracy: 0.9431 - val_loss: 0.2887 - val_accuracy: 0.8848
Epoch 5/50
30/30 [==============================] - 1s 22ms/step - loss: 0.1330 - accuracy: 0.9589 - val_loss: 0.2965 - val_accuracy: 0.8851
Epoch 6/50
30/30 [==============================] - 1s 24ms/step - loss: 0.1131 - accuracy: 0.9646 - val_loss: 0.3252 - val_accuracy: 0.8799
Epoch 7/50
30/30 [==============================] - 1s 24ms/step - loss: 0.0857 - accuracy: 0.9761 - val_loss: 0.3255 - val_accuracy: 0.8833
Epoch 8/50
30

So we could apply the early stopping method as shown below

**Practice**

In [8]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=0.9,
    restore_best_weights=True,
    start_from_epoch=0,
)
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_practice = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4, callbacks=[callback])

TypeError: EarlyStopping.__init__() got an unexpected keyword argument 'start_from_epoch'

In [ ]:
len(history_practice.history['loss'])

From the experiments above we could saw that the early stopping take action in the epoch 36 with the accuracy of 0.9999. This is our best results so far and it does save a lot of computing resources

### Regularizing your model

#### Reducing the network's size

**Original model**

In [9]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 34ms/step - loss: 0.5073 - accuracy: 0.7855 - val_loss: 0.4065 - val_accuracy: 0.8364
Epoch 2/20
30/30 [==============================] - 1s 21ms/step - loss: 0.3010 - accuracy: 0.9025 - val_loss: 0.2998 - val_accuracy: 0.8901
Epoch 3/20
30/30 [==============================] - 1s 24ms/step - loss: 0.2237 - accuracy: 0.9269 - val_loss: 0.2797 - val_accuracy: 0.8927
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1768 - accuracy: 0.9438 - val_loss: 0.2996 - val_accuracy: 0.8799
Epoch 5/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1432 - accuracy: 0.9551 - val_loss: 0.2827 - val_accuracy: 0.8876
Epoch 6/20
30/30 [==============================] - 1s 20ms/step - loss: 0.1184 - accuracy: 0.9635 - val_loss: 0.3276 - val_accuracy: 0.8769
Epoch 7/20
30/30 [==============================] - 1s 22ms/step - loss: 0.1014 - accuracy: 0.9691 - val_loss: 0.3167 - val_accuracy: 0.8807
Epoch 8/20
30

**Version of the model with lower capacity**

In [10]:
model = keras.Sequential([
    layers.Dense(4, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_smaller_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 31ms/step - loss: 0.5736 - accuracy: 0.7828 - val_loss: 0.4837 - val_accuracy: 0.8560
Epoch 2/20
30/30 [==============================] - 1s 18ms/step - loss: 0.4201 - accuracy: 0.8835 - val_loss: 0.4035 - val_accuracy: 0.8594
Epoch 3/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3347 - accuracy: 0.9034 - val_loss: 0.3402 - val_accuracy: 0.8813
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2773 - accuracy: 0.9184 - val_loss: 0.3085 - val_accuracy: 0.8886
Epoch 5/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2372 - accuracy: 0.9284 - val_loss: 0.2893 - val_accuracy: 0.8912
Epoch 6/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2068 - accuracy: 0.9375 - val_loss: 0.2788 - val_accuracy: 0.8900
Epoch 7/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1827 - accuracy: 0.9442 - val_loss: 0.2754 - val_accuracy: 0.8910
Epoch 8/20
30

**Version of the model with higher capacity**

In [11]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_larger_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 42ms/step - loss: 0.5490 - accuracy: 0.7599 - val_loss: 0.4879 - val_accuracy: 0.7657
Epoch 2/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2621 - accuracy: 0.8955 - val_loss: 0.3183 - val_accuracy: 0.8639
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1517 - accuracy: 0.9401 - val_loss: 0.3103 - val_accuracy: 0.8682
Epoch 4/20
30/30 [==============================] - 1s 18ms/step - loss: 0.0810 - accuracy: 0.9745 - val_loss: 0.3729 - val_accuracy: 0.8851
Epoch 5/20
30/30 [==============================] - 1s 19ms/step - loss: 0.0846 - accuracy: 0.9813 - val_loss: 0.3016 - val_accuracy: 0.8816
Epoch 6/20
30/30 [==============================] - 1s 20ms/step - loss: 0.0087 - accuracy: 0.9997 - val_loss: 0.4964 - val_accuracy: 0.8865
Epoch 7/20
30/30 [==============================] - 1s 19ms/step - loss: 9.1925e-04 - accuracy: 0.9999 - val_loss: 0.6059 - val_accuracy: 0.8830
Epoch 8/2

#### Adding weight regularization

**Adding L2 weight regularization to the model**

In [12]:
model = keras.Sequential([
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_l2_reg = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 33ms/step - loss: 0.5821 - accuracy: 0.7895 - val_loss: 0.4463 - val_accuracy: 0.8726
Epoch 2/20
30/30 [==============================] - 1s 19ms/step - loss: 0.3845 - accuracy: 0.8975 - val_loss: 0.3786 - val_accuracy: 0.8881
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.3255 - accuracy: 0.9151 - val_loss: 0.3844 - val_accuracy: 0.8763
Epoch 4/20
30/30 [==============================] - 1s 24ms/step - loss: 0.2904 - accuracy: 0.9283 - val_loss: 0.3720 - val_accuracy: 0.8801
Epoch 5/20
30/30 [==============================] - 1s 24ms/step - loss: 0.2752 - accuracy: 0.9320 - val_loss: 0.3735 - val_accuracy: 0.8794
Epoch 6/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2601 - accuracy: 0.9403 - val_loss: 0.3626 - val_accuracy: 0.8869
Epoch 7/20
30/30 [==============================] - 1s 22ms/step - loss: 0.2537 - accuracy: 0.9415 - val_loss: 0.3738 - val_accuracy: 0.8827
Epoch 8/20
30

**Different weight regularizers available in Keras**

In [13]:

regularizers.l1(0.001)
regularizers.l1_l2(l1=0.001, l2=0.001)

#### Adding dropout

**Adding dropout to the IMDB model**

In [14]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_dropout = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 38ms/step - loss: 0.6334 - accuracy: 0.6313 - val_loss: 0.5263 - val_accuracy: 0.8450
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.4957 - accuracy: 0.7790 - val_loss: 0.3942 - val_accuracy: 0.8697
Epoch 3/20
30/30 [==============================] - 1s 20ms/step - loss: 0.4059 - accuracy: 0.8381 - val_loss: 0.3238 - val_accuracy: 0.8837
Epoch 4/20
30/30 [==============================] - 1s 21ms/step - loss: 0.3348 - accuracy: 0.8758 - val_loss: 0.2905 - val_accuracy: 0.8889
Epoch 5/20
30/30 [==============================] - 1s 24ms/step - loss: 0.2883 - accuracy: 0.8965 - val_loss: 0.2742 - val_accuracy: 0.8920
Epoch 6/20
30/30 [==============================] - 1s 25ms/step - loss: 0.2492 - accuracy: 0.9133 - val_loss: 0.2875 - val_accuracy: 0.8846
Epoch 7/20
30/30 [==============================] - 1s 24ms/step - loss: 0.2153 - accuracy: 0.9266 - val_loss: 0.2786 - val_accuracy: 0.8900
Epoch 8/20
30

## Summary